In [17]:
import cv2
import numpy as np
from OpenCV_Yolo import *
from modules.clova_text_recognition import predict

In [18]:
yolo_v4, classes = load_yolo("yolov4_tiny/cfg/yolov4-tiny-obj.cfg", 
                    "yolov4_tiny/backup/yolov4-tiny-obj_best.weights", 
                    "yolov4_tiny/data/obj.names")
ln = yolo_v4.getLayerNames()
ln = [ln[i[0] - 1] for i in yolo_v4.getUnconnectedOutLayers()]
colors = {"plate": (0, 255, 0)}
print(classes)

In [19]:
wait = [0, 30, 15, 1]
cap = cv2.VideoCapture("Videos/6.mp4")
t = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    ab_boxes, confidences, classIDs, idxs = make_prediction(
        yolo_v4, ln, classes, frame, 0.35 , 0.35)
    output = convert_ouput(ab_boxes, classes, confidences, classIDs, idxs, colors, [])
    for out in output:
        x1, y1, x2, y2 = out["pos"]
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        crop = frame[y1:y2, x1:x2]
        if crop.shape[0]*crop.shape[1]>0:
            text = predict([crop])[0]
            cv2.putText(frame, text, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        
    cv2.imshow("frame", frame)
    k = cv2.waitKey(wait[t])
    if k == ord("p"):
        t = (t+1)%len(wait)
    if k==32:
        t = 0
    if k == ord('q'):
        break
cv2.destroyAllWindows()

In [1]:
%config IPCompleter.greedy=False